In [ ]:
from imutils import paths
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import train_test_split
import random

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
tf.config.list_physical_devices('GPU')

#**Fetching**

In [ ]:
# Reading FRI Images
imagePaths = list(paths.list_images(r"/media/gunner/5A300FFA300FDC3F/My Projects/Temp/Data-Mining/Dataset/FR0"))

data = []
data_cropped = []
labels = []
label = 0
counter_0 = 0

for imagePath in imagePaths:
  image = tf.io.read_file(imagePath)
  image = tf.image.decode_image(image, channels=1)
  data.append(image)
  image = tf.image.crop_to_bounding_box(image, 124, 124, 50, 50)
  data_cropped.append(image)
  labels.append(label)
  counter_0 += 1
print(f'Loaded FR0... {counter_0} files has loaded.')

In [ ]:
# Reading FRI Images
imagePaths = list(paths.list_images(r"/media/gunner/5A300FFA300FDC3F/My Projects/Temp/Data-Mining/Dataset/FRI"))


label = 1
counter_1 = 0

for imagePath in imagePaths:
  image = tf.io.read_file(imagePath)
  image = tf.image.decode_image(image, channels=1)
  data.append(image)
  image = tf.image.crop_to_bounding_box(image, 124, 124, 50, 50)
  data_cropped.append(image)
  labels.append(label)
  counter_1 += 1
print(f'Loaded FRI... {counter_1} files has loaded.')

In [ ]:
# Reading FRI Images
imagePaths = list(paths.list_images(r"/media/gunner/5A300FFA300FDC3F/My Projects/Temp/Data-Mining/Dataset/FRII"))

label = 2
counter_2 = 0

for imagePath in imagePaths:
  image = tf.io.read_file(imagePath)
  image = tf.image.decode_image(image, channels=1)
  data.append(image)
  image = tf.image.crop_to_bounding_box(image, 124, 124, 50, 50)
  data_cropped.append(image)
  labels.append(label)
  counter_2 += 1
print(f'Loaded FRII... {counter_2} files has loaded.')

#**EDA**

In [ ]:
counter = Counter(labels)
sorted_counts = dict(sorted(counter.items()))
plt.bar(sorted_counts.keys(), sorted_counts.values(), color='skyblue')
plt.xlabel('Class')
plt.ylabel('Count')
plt.title('Count of Each Class in the Dataset')
plt.xticks(range(3))
plt.show()

In [8]:
labels = np.array(labels,dtype='uint8')
data = np.array(data,dtype='uint8')
data_cropped = np.array(data_cropped,dtype='uint8')

In [ ]:
image = np.squeeze(data[0])
plt.subplot(1, 3, 1)
plt.imshow(image, cmap='gray')
plt.axis('off')
plt.title(f"Image at FR0")

image = np.squeeze(data[6066])
plt.subplot(1, 3, 2)
plt.imshow(image, cmap='gray')
plt.axis('off')
plt.title(f"Image at FRI")

image = np.squeeze(data[11075])
plt.subplot(1, 3, 3)
plt.imshow(image, cmap='gray')
plt.axis('off')
plt.title(f"Image at FRII")


plt.show()

In [ ]:
image = np.squeeze(data_cropped[0])
plt.subplot(1, 3, 1)
plt.imshow(image, cmap='gray')
plt.axis('off')
plt.title(f"Image at FR0")

image = np.squeeze(data_cropped[6066])
plt.subplot(1, 3, 2)
plt.imshow(image, cmap='gray')
plt.axis('off')
plt.title(f"Image at FRI")

image = np.squeeze(data_cropped[11075])
plt.subplot(1, 3, 3)
plt.imshow(image, cmap='gray')
plt.axis('off')
plt.title(f"Image at FRII")


plt.show()

In [ ]:
data.shape, data_cropped.shape

In [ ]:
np.min(data_cropped[0]), np.max(data_cropped[0])

#**Preprocessing**

In [13]:
data_normalized = data_cropped / 255

In [ ]:
print(f'Min of original = {np.min(data_cropped[0])}')
print(f'Max of original = {np.max(data_cropped[0])}')
print(f'Min of original = {np.min(data_normalized[0])}')
print(f'Max of original = {np.max(data_normalized[0])}')

In [ ]:
cropped_flat = data_cropped.flatten()
normalized_flat = data_normalized.flatten()

plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.boxplot(cropped_flat)
plt.title("Original Pixel Values (0-255)")
plt.ylabel("Pixel Intensity")

plt.subplot(1, 2, 2)
plt.boxplot(normalized_flat)
plt.title("Normalized Pixel Values (0-1)")
plt.ylabel("Pixel Intensity")

plt.tight_layout()
plt.show()

#**Splitting**

In [ ]:
x = data_normalized
y = labels
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

x_train.shape, y_train.shape, x_test.shape, y_test.shape

#**Model definition**

In [17]:
model = keras.Sequential()
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(units=64, activation='relu'))
model.add(keras.layers.Dense(units=64, activation='relu'))
model.add(keras.layers.Dense(units=64, activation='relu'))
model.add(keras.layers.Dense(units=64, activation='relu'))
model.add(keras.layers.Dense(units=64, activation='relu'))
model.add(keras.layers.Dense(units=32, activation='relu'))

model.add(keras.layers.Dense(units=3, activation=keras.activations.softmax))

In [18]:
optimizer = keras.optimizers.Adam()
loss = keras.losses.SparseCategoricalCrossentropy()
metric = keras.metrics.CategoricalAccuracy()

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
model.build(input_shape=(None, 50, 50))
model.summary()

In [ ]:
hist = model.fit(x_train, y_train, batch_size=32, epochs=20, validation_data=(x_test, y_test))

In [ ]:
plt.axes().set_facecolor('gray')
plt.grid('True')

loss = hist.history['loss']
val_loss = hist.history['val_loss']
plt.plot(loss, color='green')
plt.plot(val_loss, color='yellow')

acc_train = hist.history['accuracy']
val_acc_test = hist.history['val_accuracy']
plt.plot(acc_train, color='blue')
plt.plot(val_acc_test, color='red')



plt.show()


In [ ]:
y_pred_train = model.predict(x_train)
y_pred_test = model.predict(x_test)

index_train = random.randint(0, 9198)
print(f'y_train: {y_train[index_train]} y_pred_train:{y_pred_train[index_train]}')

index_test = random.randint(0, 3948)
print(f'y_test: {y_test[index_test]} y_pred_test:{y_pred_test[index_test]}')

#**CNN**

In [ ]:
model_cnn = keras.Sequential()
model_cnn.add(keras.layers.Conv2D(128, kernel_size=(5,5), strides=(1, 1),padding='valid', activation='relu', input_shape=(50, 50, 1)))
model_cnn.add(keras.layers.MaxPool2D(pool_size=(2, 2)))
model_cnn.add(keras.layers.Conv2D(128, kernel_size=(3,3), strides=(1, 1),padding='valid', activation='relu'))
model_cnn.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model_cnn.add(keras.layers.Conv2D(128, kernel_size=(3,3), strides=(1, 1),padding='valid', activation='relu'))
model_cnn.add(keras.layers.MaxPool2D(pool_size=(2, 2)))
model_cnn.add(keras.layers.Conv2D(64, kernel_size=(3,3), strides=(1, 1),padding='valid', activation='relu'))
model_cnn.add(keras.layers.AveragePooling2D(pool_size=(2, 2)))
model_cnn.add(keras.layers.Conv2D(64, kernel_size=(3,3), strides=(1, 1),padding='same', activation='relu'))
model_cnn.add(keras.layers.Conv2D(64, kernel_size=(3,3), strides=(1, 1),padding='same', activation='relu'))

model_cnn.add(keras.layers.Flatten())
model_cnn.add(keras.layers.Dense(units=256, activation='relu'))
model_cnn.add(keras.layers.Dense(units=128, activation='relu'))
model_cnn.add(keras.layers.Dense(units=64, activation='relu'))
model_cnn.add(keras.layers.Dense(units=32, activation='relu'))

model_cnn.add(keras.layers.Dense(units=3, activation=keras.activations.softmax))


#Model Compiling
optimizer = keras.optimizers.Adam()
loss = keras.losses.SparseCategoricalCrossentropy()
metric = [keras.metrics.CategoricalAccuracy()]

model_cnn.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])


#Builing and Summury
model_cnn.build(input_shape=(None, 50, 50))
model_cnn.summary()

In [ ]:
hist = model_cnn.fit(x_train, y_train, batch_size=64, epochs=20, validation_data=(x_test, y_test))

In [ ]:
acc_train = hist.history['accuracy']
acc_test = hist.history['val_accuracy']
loss_train = hist.history['loss']
loss_test = hist.history['val_loss']

plt.axes().set_facecolor('gray')
plt.grid('True')


plt.plot(loss_train, color='green', label='Train loss')
plt.plot(loss_test, color='yellow', label='Test loss')
plt.plot(acc_train, color='blue', label='Train acc')
plt.plot(acc_test, color='red', label='Test acc')


plt.show()

In [ ]:
y_pred_train = model_cnn.predict(x_train)
y_pred_test = model_cnn.predict(x_test)


index_train = random.randint(0, 9198)
print(f'y_train: {y_train[index_train]} y_pred_train:{y_pred_train[index_train]}')

index_test = random.randint(0, 3942)
print(f'y_test: {y_test[index_test]} y_pred_test:{y_pred_test[index_test]}')


In [ ]:
type(y_pred_test), y_pred_test.shape, type(y_pred_train), y_pred_train.shape

In [30]:
def softmax(x):
    ex = np.exp(x - np.max(x, axis=1, keepdims=True))
    return ex
    

In [31]:
soft_y_train = softmax(y_pred_train)
y_pred_label_train = np.argmax(soft_y_train, axis=1)
y_pred_label_train = y_pred_label_train.reshape(-1,1)

soft_y_test = softmax(y_pred_test)
y_pred_label_test = np.argmax(soft_y_test, axis=1)
y_pred_label_test = y_pred_label_test.reshape(-1,1)

#**CNN by BatchNorm**

In [ ]:
model_cnn_batch = keras.Sequential()

#Conv2D + BatchNorm + Activation
model_cnn_batch.add(keras.layers.Conv2D(128, kernel_size=(3,3), strides=(1, 1),padding='valid', input_shape=(50, 50, 1)))
model_cnn_batch.add(keras.layers.BatchNormalization())
model_cnn_batch.add(keras.layers.Activation('relu'))

#Maxpooling
model_cnn_batch.add(keras.layers.MaxPool2D(pool_size=(2, 2)))


#Conv2D + BatchNorm + Activation
model_cnn_batch.add(keras.layers.Conv2D(128, kernel_size=(3,3), strides=(1, 1),padding='valid', input_shape=(50, 50, 1)))
model_cnn_batch.add(keras.layers.BatchNormalization())
model_cnn_batch.add(keras.layers.Activation('relu'))

#Maxpooling
model_cnn_batch.add(keras.layers.MaxPool2D(pool_size=(2, 2)))


#Conv2D + BatchNorm + Activation
model_cnn_batch.add(keras.layers.Conv2D(128, kernel_size=(3,3), strides=(1, 1),padding='valid', input_shape=(50, 50, 1)))
model_cnn_batch.add(keras.layers.BatchNormalization())
model_cnn_batch.add(keras.layers.Activation('relu'))

#Maxpooling
model_cnn_batch.add(keras.layers.MaxPool2D(pool_size=(2, 2)))


#Flatten
model_cnn_batch.add(keras.layers.Flatten())
model_cnn_batch.add(keras.layers.Dense(units=256, activation='relu'))

model_cnn_batch.add(keras.layers.Dense(units=3, activation=keras.activations.softmax))


#Model Compiling
optimizer = keras.optimizers.Adam()
loss = keras.losses.SparseCategoricalCrossentropy()
metric = [keras.metrics.CategoricalAccuracy()]

model_cnn_batch.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])


#Builing and Summury
model_cnn_batch.build(input_shape=(None, 50, 50))
model_cnn_batch.summary()

In [ ]:
hist = model_cnn_batch.fit(x_train, y_train, batch_size=64, epochs=20, validation_data=(x_test, y_test))

In [ ]:
acc_train = hist.history['accuracy']
acc_test = hist.history['val_accuracy']
loss_train = hist.history['loss']
loss_test = hist.history['val_loss']

plt.axes().set_facecolor('gray')
plt.grid('True')


plt.plot(loss_train, color='green', label='Train loss')
plt.plot(loss_test, color='yellow', label='Test loss')
plt.plot(acc_train, color='blue', label='Train acc')
plt.plot(acc_test, color='red', label='Test acc')


plt.show()